In [ ]:
import os
import numpy as np
import sys
import holoviews as hv

sys.path.append('/Users/annateruel/phd_code/')
from ca2img.minian_dca1 import (preprocessing,  
                                start_cluster,
                                motion_correction,
                                initialize)

from minian.utilities import (open_minian,
                              get_optimal_chk,
                              save_minian)
from minian.motion_correction import apply_transform, estimate_motion
from minian.visualization import visualize_motion

In [ ]:
video_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'

In [ ]:
hv.notebook_extension("bokeh", width=100)

# Minian Analysis in dCA1


In [ ]:
start_cluster(n_workers=8, memory_limit="6GB")

### Preprocessing

In [ ]:
preprocessing(intpath=int_dir, dpath=video_dir, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 7}, 
                          param_background_removal={"method": "tophat", "wnd": 15})

In [ ]:
dirs = ['/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-113']

for dir in dirs:
    for root, _, _ in os.walk(dir):
        if root.endswith("My_V4_Miniscope"):
            preprocessing(intpath=root, dpath=root, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 7}, 
                          param_background_removal={"method": "tophat", "wnd": 15})

### Motion correction

In [ ]:
param_estimate_motion = {"dim": "frame"}

param_save_minian = {
    "dpath": video_dir,  
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}

minian_ds = open_minian(int_dir)
varr_ref = minian_ds['varr_ref']
chk, _ = get_optimal_chk(varr_ref, dtype=float)
motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

In [ ]:
root_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-113'

param_estimate_motion = {"dim": "frame"}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        minian_ds = open_minian(int_dir)
        varr_ref = minian_ds['varr_ref']
        chk, _ = get_optimal_chk(varr_ref, dtype=float)

        param_save_minian = {
            "dpath": int_dir,  
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        %time 
        motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

### Initialization

In [ ]:
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
os.environ["MINIAN_INTERMEDIATE"] = int_dir

param_save_minian = {
    "dpath": int_dir,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}
param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8} 

varr_ref = open_minian(int_dir)['varr_ref']
chk = get_optimal_chk(varr_ref, dtype=float)

Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

initialize(Y_fm_chk, 
           Y_hw_chk, 
           param_save_minian, 
           param_seeds_init, 
           param_pnr_refine, 
           param_ks_refine, 
           param_seeds_merge, 
           param_initialize, 
           param_init_merge, 
           int_dir, 
           chk)

In [ ]:
print("A data sample:", A.isel(unit_id=0).values)
print("C data sample:", C.isel(unit_id=0).values)

In [ ]:
print("Y_fm_chk data sample:", Y_fm_chk.isel(frame=0, height=0, width=0).values)
print("C_chk data sample:", C_chk.isel(frame=0, unit_id=0).values)

In [ ]:
from holoviews.operation.datashader import regrid
from holoviews.operation.datashader import regrid, datashade
from holoviews.operation.datashader import regrid, datashade

output_size = 100
hv.output(size=int(output_size * 0.55))
im_opts = dict(
    frame_width=500,
    aspect=A.sizes["width"] / A.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
cr_opts = dict(frame_width=750, aspect=1.5 * A.sizes["width"] / A.sizes["height"])
(
    regrid(
        hv.Image(
            A.max("unit_id").rename("A").compute().astype(np.float32),
            kdims=["width", "height"],
        ).opts(**im_opts)
    ).relabel("Initial Spatial Footprints")
    + regrid(
        hv.Image(
            C.rename("C").compute().astype(np.float32), kdims=["frame", "unit_id"]
        ).opts(cmap="viridis", colorbar=True, **cr_opts)
    ).relabel("Initial Temporal Components")
).cols(2)

In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-115'

param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        param_save_minian = {
            "dpath": int_dir,
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        varr_ref = open_minian(int_dir)['varr_ref']
        chk = get_optimal_chk(varr_ref, dtype=float)

        Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
        Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

        initialize(Y_fm_chk, 
                   Y_hw_chk, 
                   param_save_minian, 
                   param_seeds_init, 
                   param_pnr_refine, 
                   param_ks_refine, 
                   param_seeds_merge, 
                   param_initialize, 
                   param_init_merge, 
                   int_dir, 
                   chk)